In [ ]:
!pip install -q gradio

In [ ]:
import os
import glob

import gradio as gr

from encoder import *
from decoder import *

MODELS_PATH = '../models/'
availableModels = [f for f in glob.glob(MODELS_PATH + "*.pt", recursive=True)]

RESOLUTION = 36
KEYBOARD_SIZE = 88
MIDI_OFFSET = 20


def getFileLen(contextFile):
  if isinstance(contextFile, gr.inputs.File):
    contextFileLen = 2
  if isinstance(contextFile, pd.DataFrame):
    contextFileLen = len(contextFile)
  else:
    contextFileLen = len(encode_data( os.path.abspath(contextFile),
                                      n_frames=RESOLUTION,
                                      n_notes=KEYBOARD_SIZE,
                                      midi_offset=MIDI_OFFSET
                                ))
    return contextFileLen

def runQuery( modelPath,
              contextFile,
              firstContextMeasure,
              lastContextMeasure,
              amountToGenerate,
              temperature):
  
  model = torch.load(modelPath)
  
  
  # mid -> pkl
  encoded_context = encode_data(contextFile,
                                n_frames=RESOLUTION,
                                n_notes=KEYBOARD_SIZE,
                                midi_offset=MIDI_OFFSET)

  # Filter by instrument
  encoded_context = encoded_context[encoded_context.index==instrument]

  # If the file does not contain the instrument
  if len(encoded_context) == 0:
      raise ValueError(f'Oops, it seems that the selected context file does not contain any ({INSTRUMENT}).')
  
  # Calclulate frames
  first_context_frame = (firstContextMeasure-1)*RESOLUTION
  last_context_frame = first_context_frame + (lastContextMeasure-1)*RESOLUTION
  first_generated_frame = last_context_frame
  last_generated_frame = first_generated_frame + amountToGenerate*RESOLUTION

  # Separate Info from stackframe
  context_info = getInfo(encoded_context, first_context_frame, last_context_frame)
  context_sf = getStackframe(encoded_context, first_context_frame, last_context_frame, to_float=True)
  generated_info = getInfo(encoded_context, first_generated_frame, last_generated_frame)

  generated_sf = generateMeasures(model,
                                  context_sf,
                                  RESOLUTION,
                                  amount=amountToGenerate,
                                  temperature=temperature
                                  )

  generated_representation = mergeData(generated_info, generated_sf, instrument, MIDI_OFFSET)

  generatedFile = decode_data(generated_representation, RESOLUTION)

  return generatedFile



# INPUTS  

modelPath = gr.inputs.Dropdown(availableModels, type="value", default=None, label='Select model')
instrument = gr.inputs.Textbox(lines=1, placeholder=None, default="", numeric=False, type="str", label="Enter model's intrument")

contextFile = gr.inputs.File(file_count="single", type="file", label='Upload MIDI context file', keep_filename=True)
firstContextMeasure = gr.inputs.Slider( minimum=1,
                                        maximum=getFileLen(contextFile),
                                        default=1,
                                        step=1,
                                        label='Select the first measure to be fed to the model')
lastContextMeasure = gr.inputs.Slider(minimum=1,
                                      maximum=getFileLen(contextFile),
                                      default=getFileLen(contextFile),
                                      step=1,
                                      label='Select the last measure to be fed to the model')

amountToGenerate = gr.inputs.Number(default=1, label='Select amount of measures to generate')
temperature = gr.inputs.Number(default=0.5, label='Select generation temperature (must be between 0 and 1)')


# OUTPUTS
generatedFile = gr.outputs.File(label='Generated result')


iface = gr.Interface( fn=runQuery,
                      inputs=[modelPath,
                              instrument,
                              contextFile,
                              firstContextMeasure,
                              lastContextMeasure,
                              amountToGenerate,
                              temperature
                      ],
                      outputs=generatedFile,
                      # live=True,
                      title='Bidirectional-LSTM based MIDI generation'
                      )
iface.launch(share=True, debug=True, inbrowser=True)

Running locally at: http://127.0.0.1:7867/
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)


Running on External URL: https://21927.gradio.app
Interface loading below...


[2021-09-02 17:24:03,835] ERROR in app: Exception on /api/predict/ [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.8/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_

[2021-09-02 17:24:30,584] ERROR in app: Exception on /api/predict/ [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.8/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_

[2021-09-02 17:24:32,525] ERROR in app: Exception on /api/predict/ [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.8/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_

[2021-09-02 17:24:33,757] ERROR in app: Exception on /api/predict/ [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.8/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_

[2021-09-02 17:24:35,059] ERROR in app: Exception on /api/predict/ [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.8/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_

[2021-09-02 17:24:37,002] ERROR in app: Exception on /api/predict/ [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.8/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_

[2021-09-02 17:24:40,995] ERROR in app: Exception on /api/predict/ [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.8/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_

[2021-09-02 17:24:42,244] ERROR in app: Exception on /api/predict/ [POST]
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/opt/conda/lib/python3.8/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/opt/conda/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/conda/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_